In [6]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [7]:
df = pd.read_csv("datasets/train.csv")

split = 0.8
df = df.sample(frac=1).dropna()
train = df[:int(0.8*df.shape[0])]
test = df[int(0.8*df.shape[0]):]

x_train, y_train = train["title"], train["label"]
x_test, y_test = test["title"], test["label"]

x_train_enc = [one_hot(words,10000) for words in x_train]
x_test_enc = [one_hot(words,10000) for words in x_test]

sent_length=10
x_train_enc_padded = pad_sequences(x_train_enc,padding='pre',maxlen=sent_length)

x_test_enc_padded = pad_sequences(x_test_enc,padding='pre',maxlen=sent_length)

In [8]:
x_train_enc_padded.shape

(14628, 10)

In [42]:
def create_model():
    model = tf.keras.Sequential([
    keras.layers.Dense(512, activation = "sigmoid", input_shape=(10, )),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1),
    ])
    model.compile(optimizer = "adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=[tf.keras.metrics.BinaryAccuracy()])
    return model

In [43]:
model = create_model()

In [44]:
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 512)               5632      
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 513       
                                                                 
Total params: 6145 (24.00 KB)
Trainable params: 6145 (24.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
checkpoint_path = "training_1/checkpoints"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)


In [46]:
model.fit(x_train_enc_padded, y_train, epochs = 20, validation_data=(x_test_enc_padded, y_test), callbacks=[cp_callback])


Epoch 1/20
450/458 [============================>.] - ETA: 0s - loss: 0.4925 - binary_accuracy: 0.7483
Epoch 1: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 0.4919 - binary_accuracy: 0.7482 - val_loss: 0.4460 - val_binary_accuracy: 0.7763
Epoch 2/20
443/458 [============================>.] - ETA: 0s - loss: 0.4605 - binary_accuracy: 0.7682
Epoch 2: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 0.4597 - binary_accuracy: 0.7691 - val_loss: 0.4409 - val_binary_accuracy: 0.7796
Epoch 3/20
442/458 [===========================>..] - ETA: 0s - loss: 0.4563 - binary_accuracy: 0.7693
Epoch 3: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 0.4566 - binary_accuracy: 0.7689 - val_loss: 0.4436 - val_binary_accuracy: 0.8023
Epoch 4/20
410/458 [=========================>....] - ETA: 0s - loss: 0.4485 - binary_accuracy: 0.7770
Epoch 4: 

In [47]:
loss, acc = model.evaluate(x_test_enc_padded,y_test, verbose=2)
print(f"model, accuracy: {100*acc:5.2f}%")

115/115 - 0s - loss: 0.4203 - binary_accuracy: 0.8015 - 72ms/epoch - 623us/step
model, accuracy: 80.15%


In [16]:
os.listdir(checkpoint_dir)

['checkpoints.data-00000-of-00001', 'checkpoint', 'checkpoints.index']

In [48]:
model_loaded = create_model()

In [49]:
loss_loaded, acc_loaded = model_loaded.evaluate(x_test_enc_padded, y_test)
print(f"Untrained model, accuracy: {100*acc_loaded:5.2f}%")


115/115 [==============================] - 0s 605us/step - loss: 1.1936 - binary_accuracy: 0.3768
Untrained model, accuracy: 37.68%


In [52]:
model_loaded.load_weights(checkpoint_path)

In [53]:
loss_loaded_trained, acc_loaded_trained = model_loaded.evaluate(x_test_enc_padded, y_test)
print(f"Trained model, accuracy: {100*acc_loaded_trained:5.2f}%")

115/115 [==============================] - 0s 570us/step - loss: 0.4203 - binary_accuracy: 0.8015
Trained model, accuracy: 80.15%
